# Second Hand Cars
## Marketplace

In [1]:
# Adjust Notebook Display
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Imports

# Generic Web Scrapers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ActionBuilder
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Generic Packages
import re
import time
import pandas as pd
import numpy as np
import random as rd
import math

In [3]:
# Import Created Custom Functions from other files
from ipynb.fs.full.ScrapingFunctions import move_right, scroll_element_menu, scroll_element_listing, move_and_click_element

Adding driver options to limit detection as much as possible, and then using Facebook marketplace link from instantiated driver with filters for listings in Malta and Gozo related to cars with a minimum price of 1,000 euros.

In [4]:
# Chrome Options
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--incognito")
options.add_argument("--disable-gpu")

# Locate Chrome Driver
service_driver = Service('chromedriver.exe')

# Add Customisation to Driver
driver = webdriver.Chrome(service=service_driver, options=options)

# Apply Stealth Function
stealth(driver,
        user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)

# Open Facebook
driver.get("https://www.facebook.com/marketplace/110612325626836/cars?minPrice=1000&exact=false")

<b>NOTE:</b> Wait for web page to load before running cell to choose cookies option.

In [5]:
# Handle Cookies - 'Only Allow Essential Cookies'
move_and_click_element('/html/body/div[2]/div[1]/div/div[2]/div/div/div/div[2]/div/div[1]/div[2]/div', driver)

Screenshot is saved to view format of listings page for the latest scraping.

In [6]:
# Get Screenshot of Results Page
time.sleep(13 + np.random.choice([x/10 for x in range(7,23)]))
driver.save_screenshot('Page_Screenshot.png')
time.sleep(17 + np.random.choice([x/10 for x in range(7,23)]))

<b>NOTE:</b> Adjust batch_count and start_count depending on if running scraper from beginning of page or continuing from previous listing.

In [7]:
# Placeholders for items to scrape
title = []
price = []
condition = []
description = []
location = []

# Set Counter
counter = 1
batch_count = 1
start_count = 1

# If not starting from beginning
if start_count != 1:
    # Check if Multiple of 3
    if (start_count % 3) != 0:
        if start_count < 3:
            start_count += 1 # If not 1 and less then 3 = 2, add 1 to be multiple of 3 (minimal loss of data)
        else:
            start_count -= 1 # Subtract by 1: First attempt at converting start_count to multiple of 3
            if (start_count % 3) != 0:
                start_count -= 1 # Second attempt at converting start_count to multiple of 3

# Scrape till Break / Interrupt
while True:
    
    # Create listing xpath
    if start_count == 13:
        # Adjustfor New Batch
        batch_count += 2
        start_count = 1
       
    # Create listing xpath
    listing_xpath = '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[2]/div/div/div[6]/div[' + str(batch_count) + ']/div[2]/div[' + str(start_count) + ']'
    
    # Display Scraper Progress
    print('Current Listing Number :', batch_count, ', ', start_count, '-> Total :', counter)
    
    # Check if listing is displayed
    try:
        if driver.find_element(By.XPATH, listing_xpath).is_displayed() == False:
            try:
                sleep_time = 33 + np.random.choice([x/10 for x in range(7,22)])
                WebDriverWait(driver, sleep_time).until(EC.presence_of_element_located((By.XPATH, listing_xpath)))
            except TimeoutException:
                print('Driver Considerably Slowing Down - Forced Shutdown')
                driver.close()
                break
    # Move to where element is displayed
    except NoSuchElementException: 
        # Keep Scrolling to the Bottom of the Page till Elemetn is Located
        # To got to element where previously left off
        try:
            while True: 
                try:
                    if driver.find_element(By.XPATH, listing_xpath).is_displayed() == True:
                        break
                except:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(np.random.choice([x/10 for x in range(7,22)]))
        # Raise Login as Issue, and others that overlay display after too long scraping
        except:
            # Re-Open Facebook to Avoid Login
            driver.get("https://www.facebook.com/marketplace/110612325626836/cars?minPrice=1000&exact=false")
            # Scroll to next element
            while True: # To got to element where previously left off
                try:
                    if driver.find_element(By.XPATH, listing_xpath).is_displayed() == True:
                        break
                except:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(np.random.choice([x/10 for x in range(7,22)]))
            scroll_element_menu(listing_xpath, driver)
            time.sleep(np.random.choice([x/10 for x in range(7,22)]))

    # Scroll to listing if listing number - 1 is a multiple of 3
    if (start_count-1 % 3) == 0:
        time.sleep(np.random.choice([x/10 for x in range(27,42)]))
        scroll_element_menu(listing_xpath, driver)
    
    # Move to listing and Open it
    time.sleep(np.random.choice([x/10 for x in range(7,22)]))
    move_and_click_element(listing_xpath, driver)
    
    # Open listing and scrape data
    
    time.sleep(3 + np.random.choice([x/10 for x in range(7,23)]))
    
    # Check if listing takes significant time to open
    try:
        if driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]').is_displayed() == False:
            sleep_time = 60 + np.random.choice([x/10 for x in range(7,22)])
            WebDriverWait(driver, sleep_time).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]')))
    except:
        print('Driver Considerably Slowing Down - Forced Shutdown')
        driver.close()
        break
       
    # Get Headline Text
    title.append(driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]').text)
    time.sleep(np.random.choice([x/10 for x in range(17,33)]))
    
    # Get Price
    pr = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[1]/div[2]').text
    if '$' in pr:
        price.append(int(pr.replace('$','').replace(',',''))*0.9) # Adjust for USD-EUR Exchange Rate in 04/2023
    if '€' in pr:
        price.append(int(pr.replace('€','').replace(',','')))
    else:
        price.append(np.nan)
    time.sleep(np.random.choice([x/10 for x in range(17,33)]))
    
    # Get Condition
    try:
        condition.append(driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/ul/li/div/span[2]').text)
    except NoSuchElementException:
        condition.append(np.nan)
    time.sleep(np.random.choice([x/10 for x in range(17,33)]))
    
    # Get Description
    move_right(driver) # Move to right panel before scrolling
    scroll_element_listing('/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[1]', driver) # Scroll to description
    try: # Check if 'See More' Button is displayed/enabled
        move_and_click_element('/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[1]/div/span/div', driver)
        time.sleep(np.random.choice([x/10 for x in range(7,22)]))
        scroll_element_listing('/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[1]/div/span/div', driver)
    except NoSuchElementException:
        pass
    description.append(driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[1]').text.replace('\n',' '))
    time.sleep(np.random.choice([x/10 for x in range(17,33)]))
    
    # Get Location
    move_right(driver) # Move to right panel before scrolling
    try:
        scroll_element_listing('/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[3]/div/div/div[1]', driver) # Scroll to location
        location.append(driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[3]/div/div/div[1]').text)
    except:
        scroll_element_listing('/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[2]/div/div/div[1]', driver) # Scroll to location
        location.append(driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[1]/div[5]/div[2]/div/div[2]/div/div/div[1]').text)
    time.sleep(np.random.choice([x/10 for x in range(17,33)]))
    
    # Move to close button and click
    move_and_click_element('/html/body/div[1]/div/div[1]/div/div[2]/div/div[1]/div/div[1]/div', driver)
    
    time.sleep(np.random.choice([x/10 for x in range(37,55)]))
    
    start_count += 1
    counter += 1
    
driver.quit()
print('Scraping Done!')

Current Listing Number : 1 ,  1 -> Total : 1
Current Listing Number : 1 ,  2 -> Total : 2
Current Listing Number : 1 ,  3 -> Total : 3
Current Listing Number : 1 ,  4 -> Total : 4
Current Listing Number : 1 ,  5 -> Total : 5
Current Listing Number : 1 ,  6 -> Total : 6
Current Listing Number : 1 ,  7 -> Total : 7
Current Listing Number : 1 ,  8 -> Total : 8
Current Listing Number : 1 ,  9 -> Total : 9
Current Listing Number : 1 ,  10 -> Total : 10
Current Listing Number : 1 ,  11 -> Total : 11
Current Listing Number : 1 ,  12 -> Total : 12
Current Listing Number : 3 ,  1 -> Total : 13
Current Listing Number : 3 ,  2 -> Total : 14
Current Listing Number : 3 ,  3 -> Total : 15
Current Listing Number : 3 ,  4 -> Total : 16
Current Listing Number : 3 ,  5 -> Total : 17
Current Listing Number : 3 ,  6 -> Total : 18
Current Listing Number : 3 ,  7 -> Total : 19
Current Listing Number : 3 ,  8 -> Total : 20
Current Listing Number : 3 ,  9 -> Total : 21
Current Listing Number : 3 ,  10 -> Tota

Current Listing Number : 29 ,  7 -> Total : 175
Current Listing Number : 29 ,  8 -> Total : 176
Current Listing Number : 29 ,  9 -> Total : 177
Current Listing Number : 29 ,  10 -> Total : 178
Current Listing Number : 29 ,  11 -> Total : 179
Current Listing Number : 29 ,  12 -> Total : 180
Current Listing Number : 31 ,  1 -> Total : 181
Current Listing Number : 31 ,  2 -> Total : 182
Current Listing Number : 31 ,  3 -> Total : 183
Current Listing Number : 31 ,  4 -> Total : 184
Current Listing Number : 31 ,  5 -> Total : 185
Current Listing Number : 31 ,  6 -> Total : 186
Current Listing Number : 31 ,  7 -> Total : 187
Current Listing Number : 31 ,  8 -> Total : 188
Current Listing Number : 31 ,  9 -> Total : 189
Current Listing Number : 31 ,  10 -> Total : 190
Current Listing Number : 31 ,  11 -> Total : 191
Current Listing Number : 31 ,  12 -> Total : 192
Current Listing Number : 33 ,  1 -> Total : 193
Current Listing Number : 33 ,  2 -> Total : 194
Current Listing Number : 33 ,  3 -

In [8]:
# Get scraped data into dataframe format

# Create empty dataframe
marketplace = pd.DataFrame()

# Populate empty dataframe with scraped data
marketplace['Title'] = title
marketplace['Price'] = price
marketplace['Locality'] = location
marketplace['Description'] = description
marketplace['Condition'] = condition

# View dataframe
print('DataFrame Size :', marketplace.shape)
marketplace.head()

DataFrame Size : (218, 5)


,Title,Price,Locality,Description,Condition
0,FORD FIESTA,2800,Birkirkara,Ford fiesta ta Malta Manual Year 2007 1.4 p...,Used - Good
1,Mazda Demio - 2010 - Auto - Serviced - Read De...,5999,Żejtun,✅ Mazda ✅ Demio ✅ Hatchback ✅ 2010 ✅ Automatic...,Used - like new
2,Suzuki swift,4800,Birkirkara,Suzuki swift in good condition 2008,NaN
3,2005 Daihatsu sirion,2800,Birżebbuġa,DAIHATSU SIRION YEAR OF MANUFACTURE : 2005 ...,NaN
4,Toyota Ractis 2005 (1.3 petrol),4400,Żejtun,Toyota Ractis Japan Import 1.3 Petrol Milage 1...,NaN


Several short scraping sessions were made during a period of several days, and data gather from previous sessions are added to the latest. This is considered reasonable given the little to no distinction in the nature of listings on Facebook Marketplace across several days. If scraping occured over longer time periods the time dimension would have to be taken care of, perhaps with a day of scraping variable.

In [9]:
# Load Previously Scraped Data if Continuing Process from Previous Scraping
marketplace_prev = pd.read_csv('marketplace_cars.csv')

# Join old and new datasets
marketplace = pd.concat([marketplace_prev, marketplace])
print('New Cumulative DataFrame Size:', marketplace.shape)

New Cumulative DataFrame Size: (799, 5)


In [10]:
# Drop Duplicates
marketplace = marketplace.drop_duplicates()
print('DataFrame Size without Duplicates:', marketplace.shape)

DataFrame Size without Duplicates: (682, 5)


In [11]:
# Quick check of dataframe
marketplace.describe(include = 'all')

,Title,Price,Locality,Description,Condition
count,682,6.800000e+02,682,663,206
unique,588,NaN,52,607,3
top,Car,NaN,Żebbuġ,,Used - Good
freq,6,NaN,54,9,110
mean,NaN,1.923705e+05,NaN,NaN,NaN
std,NaN,1.263828e+06,NaN,NaN,NaN
min,NaN,1.000000e+03,NaN,NaN,NaN
25%,NaN,2.800000e+03,NaN,NaN,NaN
50%,NaN,4.800000e+03,NaN,NaN,NaN
75%,NaN,8.800000e+03,NaN,NaN,NaN


In [12]:
# Save Main Page Dataframe
marketplace.to_csv('marketplace_cars.csv', index=False)